# Iris dataset
* Get sample data [here](https://github.com/fuyu-quant/IBLM/tree/main/datasets).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fuyu-quant/IBLM/blob/main/examples/iblmodel_iris.ipynb)

In [1]:
%%capture
!pip install git+https://github.com/fuyu-quant/IBLM.git

In [2]:
import pkg_resources
print(pkg_resources.get_distribution('IBLM').version)

0.0.15


### Training

In [8]:
import pandas as pd
from langchain.llms import OpenAI
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from iblm import IBLMClassifier


import os
#os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

In [9]:
#df = pd.read_csv('/content/irisdata_train.csv')
df = pd.read_csv('../datasets/irisdata_train.csv')
x_train = df.drop('target', axis=1)
y_train = df['target']

In [10]:
llm_model_name = 'gpt-4'

params = {
    'columns_name': True
    }

iblm = IBLMClassifier(llm_model_name=llm_model_name, params=params)

In [11]:
#file_path = '/content/'
file_path = '../datasets/'

model = iblm.fit(x_train, y_train, model_name = 'titanic', file_path=file_path)

> Start of model creating.


UnboundLocalError: local variable 'output_code' referenced before assignment

In [25]:
# Code of the model created
print(model)

import numpy as np
def predict(x):
    df = x.copy()
    output = []
    for index, row in df.iterrows():
        # Please describe the process required to make the prediction below.

        # If the passenger is a female and in first or second class, predict survival
        if row['sex'] == 'female' and (row['pclass'] == 1 or row['pclass'] == 2):
            y = 1
        # If the passenger is a child (age <= 15) and in first or second class, predict survival
        elif row['age'] <= 15 and (row['pclass'] == 1 or row['pclass'] == 2):
            y = 1
        # If the passenger is a male and in third class, predict non-survival
        elif row['sex'] == 'male' and row['pclass'] == 3:
            y = 0
        # If the passenger is an adult male and traveling alone, predict non-survival
        elif row['adult_male'] and row['alone']:
            y = 0
        # For all other cases, predict non-survival
        else:
            y = 0

        y = 1 / (1 + np.exp(-y))

        out

## Prediction

In [26]:
#df = pd.read_csv('/content/irisdata_test.csv')
df = pd.read_csv('../datasets/irisdata_test.csv')
x_test = df.drop('survived', axis=1)
y_test = df['survived']

In [27]:
y_proba = iblm.predict(x_test)
y_pred = (y_proba > 0.5).astype(int)
#y_proba

In [28]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Precision
precision = precision_score(y_test, y_pred)
print(f'Precision: {precision}')

# Recall
recall = recall_score(y_test, y_pred)
print(f'Recall: {recall}')

# F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 score: {f1}')

# ROC-AUC (you need prediction probabilities for this, not just class predictions)
# Here we just reuse y_pred for simplicity
roc_auc = roc_auc_score(y_test, y_proba)
print(f'ROC-AUC: {roc_auc}')

Accuracy: 0.8079877112135176
Precision: 0.958904109589041
Recall: 0.5405405405405406
F1 score: 0.691358024691358
ROC-AUC: 0.7626172090457806


## Prediction from external files


In [12]:
from model_code import titanic

y_proba = titanic.predict(x_test)
y_pred = (y_proba > 0.5).astype(int)

In [13]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Precision
precision = precision_score(y_test, y_pred)
print(f'Precision: {precision}')

# Recall
recall = recall_score(y_test, y_pred)
print(f'Recall: {recall}')

# F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 score: {f1}')

# ROC-AUC (you need prediction probabilities for this, not just class predictions)
# Here we just reuse y_pred for simplicity
roc_auc = roc_auc_score(y_test, y_proba)
print(f'ROC-AUC: {roc_auc}')

Accuracy: 0.45
Precision: 0.3956043956043956
Recall: 1.0
F1 score: 0.5669291338582677
ROC-AUC: 0.9197048611111112


## Interpretation of results

In [14]:
description = iblm.interpret()

Tokens Used: 896
	Prompt Tokens: 341
	Completion Tokens: 555
Successful Requests: 1
Total Cost (USD): $0.04353


In [15]:
print(description)

- First, the function `predict` takes a DataFrame `x` as input and creates a copy of it named `df`. The columns of `df` are then renamed to be integer indices.

- The function then initializes an empty list called `output` to store the predictions.

- For each row in the DataFrame `df`, the function extracts the following features:
  - `pclass`: Passenger class (First, Second, or Third)
  - `sex`: Gender of the passenger (male or female)
  - `age`: Age of the passenger
  - `fare`: Ticket fare paid by the passenger
  - `embarked`: Port of embarkation (C, Q, or S)
  - `alone`: Whether the passenger is traveling alone or not (True or False)

- The prediction logic is then applied to these features, and a variable `y` is initialized to 0.

- The following conditions are checked and the corresponding values are added to `y`:
  - If `pclass` is 'First', add 0.3 to `y`.
  - If `pclass` is 'Second', add 0.15 to `y`.
  - If `sex` is 'female', add 0.35 to `y`.
  - If `age` is less than or equal 